In [6]:
# from google.colab import drive

# drive.mount("/content/drive")

In [7]:
import sys
import os

# Add your Google Drive path to Python path
# drive_path = "/content/drive/MyDrive/NumEval"  # Update this path
drive_path = "/content"
sys.path.append(drive_path)

In [8]:
!pip install gcsfs==2024.9.0
requirement_path = f"{drive_path}/colab_requirements.txt"
!pip install -r {requirement_path}

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [11]:
import torch
import gc
from instruction_tuning_qqa import run
from argparse import Namespace


def run_experiment(
    data_train_pth,
    data_dev_pth,
    data_test_pth,
    has_demonstrations,
    is_digit_base,
    is_text_base,
):
    try:
        # Update paths for Colab
        output_model_path = f"{drive_path}/models/{data_train_pth.split('/')[-1].split('.')[0]}_demonstrations_{has_demonstrations}_digit_{is_digit_base}_text{is_text_base}"
        output_file_name = f"results_{data_train_pth.split('/')[-1].split('.')[0]}_demonstrations_{has_demonstrations}_digit_{is_digit_base}_text_{is_text_base}.json"

        args = Namespace(
            data_train_pth=data_train_pth,
            data_dev_pth=data_dev_pth,
            data_test_pth=data_test_pth,
            num_splits=10,
            select_split_idx=2,
            is_digit_base=is_digit_base,
            has_demonstrations=has_demonstrations,
            model_name="google/flan-t5-base",
            seed=42,
            model_checkpoint="",
            task="train",
            evaluation_strategy="epoch",
            save_strategy="no",
            per_device_train_batch_size=10,
            per_device_eval_batch_size=10,
            lr=5e-7,
            warm_up_radio=0.1,
            gradient_accumulation_steps=1,
            num_train_epochs=30,
            output_model_path=output_model_path,
            weight_decay=0.01,
            output_file_name=output_file_name,
            output_dir=f"{drive_path}/save_res",
            is_text_base=is_text_base,
            has_dev=True,
        )

        run(args)  # Execute the model training/testing pipeline

    finally:
        # Ensure cleanup in any case
        if "args" in locals():
            del args  # Remove args variable

        # Manually delete any additional objects in run() (like model if instantiated)
        # Assuming run loads models, you can check their namespace or module content
        if torch.cuda.is_available():
            torch.cuda.empty_cache()  # Clear unused GPU memory
            torch.cuda.synchronize()  # Wait for all operations to finish

        gc.collect()  # Force garbage collection

        # Logging to confirm cleanup
        print("Memory cleanup completed after iteration.")


def automate_experiments():

    data_folder_pth = f"{drive_path}/converted_jsons/QQA"
    data_train_pth = f"{data_folder_pth}/QQA_train_converted.json"
    data_dev_pth = f"{data_folder_pth}/QQA_dev_converted.json"
    data_test_pth = f"{data_folder_pth}/QQA_test_converted.json"

    demonstrations_options = [True, False]

    # demonstrations_options = [False]

    # digit_base_options = [True, False]

    digit_base_options = [False]
    is_text_base = True

    for has_demonstrations in demonstrations_options:

        for is_digit_base in digit_base_options:

            print(
                f"Running experiment with {data_train_pth}, has_demonstrations={has_demonstrations}, is_digit_base={is_digit_base}, is_text_base={is_text_base}"
            )

            run_experiment(
                data_train_pth,
                data_dev_pth,
                data_test_pth,
                has_demonstrations,
                is_digit_base,
                is_text_base,
            )

In [ ]:
automate_experiments()

Running experiment with /content/converted_jsons/QQA/QQA_train_converted.json, has_demonstrations=True, is_digit_base=False, is_text_base=True


Map:   0%|          | 0/564 [00:00<?, ? examples/s]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
